In [1]:
import sys
import os
import codecs
from math import pow, sqrt

In [2]:
users = {}
movies = {}
base_file = 'u1.base'
file = open(base_file,'r')
for line in file.readlines():
    line = line.strip().split()
    if not line[0] in users.keys():
        users[line[0]] = {line[1]:line[2]}
    else:
        users[line[0]][line[1]] = line[2]
        
    if not line[1] in movies.keys():
        movies[line[1]] = {line[0]:line[2]}
    else:
        movies[line[1]][line[0]] = line[2]


In [3]:
test = {}
test_file = 'u1.test'
file = open(test_file,'r')
for line in file.readlines():
    line = line.strip().split()
    if not line[0] in test.keys():
        test[line[0]] = {line[1]:line[2]}
    else:
        test[line[0]][line[1]] = line[2]

In [4]:
item = {}
mv_type = []

type_file = 'u.genre'
t_file = open(type_file,"r")
for t in t_file.readlines():
    t = t.strip().split('|')
    mv_type.append(t[0])
mv_type.pop()

item_file = 'u.item'
files = open(item_file, "r",  encoding="iso-8859-15")
for line in files.readlines():
    line = line.strip().split('|')
    item[line[0]] = [line[1], line[2],line[4]]
    type_list = line[5:]
    for i in range(len(mv_type)):
        if type_list[i] == '1':
            item[line[0]].append(mv_type[i])
        

In [5]:
item['1']

['Toy Story (1995)',
 '01-Jan-1995',
 'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
 'Animation',
 "Children's",
 'Comedy']

In [7]:
def matrix(mv1,mv2):
    count = 0
    for user1 in movies[mv1]:
        for user2 in movies[mv2]:
            if user1 == user2:
                count += 1
    return count

In [8]:
mv_matrix = {}
for i in movies:
    for j in movies:
        if i != j:
            count = matrix(i,j)
            if count != 0:
                mv_matrix[i] = {j:count}

In [9]:
mv_matrix

{'1': {'1682': 1},
 '2': {'1682': 1},
 '3': {'1682': 1},
 '4': {'1682': 1},
 '5': {'1682': 1},
 '7': {'1682': 1},
 '8': {'1681': 1},
 '9': {'1682': 1},
 '11': {'1682': 1},
 '13': {'1677': 1},
 '15': {'1681': 1},
 '16': {'1658': 2},
 '18': {'1651': 1},
 '19': {'1681': 1},
 '21': {'839': 1},
 '22': {'1682': 1},
 '25': {'1681': 1},
 '26': {'1672': 1},
 '28': {'1682': 1},
 '29': {'1681': 1},
 '30': {'1682': 1},
 '32': {'1677': 1},
 '34': {'1621': 1},
 '35': {'1643': 1},
 '37': {'1555': 1},
 '38': {'839': 2},
 '40': {'1664': 1},
 '41': {'1664': 1},
 '42': {'1682': 1},
 '43': {'1681': 1},
 '45': {'1674': 1},
 '46': {'1682': 1},
 '48': {'1682': 1},
 '50': {'1682': 1},
 '52': {'1682': 1},
 '55': {'1682': 1},
 '57': {'1672': 1},
 '58': {'1682': 1},
 '59': {'1672': 1},
 '63': {'839': 1},
 '66': {'1682': 1},
 '68': {'1682': 1},
 '71': {'1682': 1},
 '75': {'1655': 1},
 '77': {'1682': 1},
 '79': {'1682': 1},
 '83': {'1682': 1},
 '87': {'1682': 1},
 '88': {'1682': 1},
 '89': {'1682': 1},
 '93': {'16

In [10]:
def simularity(data,user1,user2):
    
    user1_data = data[user1]
    user2_data = data[user2]
    distance = 0
    for key in user1_data.keys():
        if key in user2_data.keys():
            distance += pow(float(user1_data[key])-float(user2_data[key]),2)
 
    return 1/(1+sqrt(distance))

In [11]:
simularity(users,'196','13')

0.1380262631157473

In [12]:
def sim_cosine(prefs, item1, item2):
    si = {}
    for i in prefs[item1]:
        if i in prefs[item2]:
            si[i] = 1

    if len(si) == 0: return 0
    x = sqrt(sum([float(prefs[item1][it]) ** 2 for it in si]))
    y = sqrt(sum([float(prefs[item2][it]) ** 2 for it in si]))
    xy = sum([float(prefs[item1][it]) * float(prefs[item2][it]) for it in si])
    cos = xy / (x * y)
    return cos

In [165]:
sim_cosine(users,'1','589')

1.0

In [166]:
def top5_simliar(userId):
    
    res = {}
    movie = users[userId]
    if len(movie) == 0:
        return []
    nbhd = []
    for mv in movie:
        for pp in movies[mv]:
            nbhd.append(pp)
    nbhd = list(set(nbhd))
    for user in nbhd:
        res[user] = sim_cosine(users,userId,user)
    
    res = sorted(res, key=lambda key: res[key],reverse=True)
    
    return res[:5]

In [167]:
top5_simliar('196')

['656', '319', '163', '557', '81']

In [168]:
def popular_movie(movies):
    
    ppl_mv = {}
    mmax = 0
    for mv in movies:
        ppl_mv[mv] = len(movies[mv])
    ppl_mv = sorted(ppl_mv, key=lambda key: ppl_mv[key],reverse=True)
    

    return ppl_mv

In [169]:
popular_movie(movies)[:10]

['50', '181', '258', '100', '294', '288', '286', '1', '121', '300']

In [13]:
def mv_recommend(userId):
    if userId not in users:
        mv_list = popular_movie(movies)
        mv_remd = mv_list[:10]
    else:
        recommend = {}
        mv_list = users[userId]
        mv_list = sorted(mv_list, key=lambda key: mv_list[key],reverse=True)
        for mv in movies:
            if mv not in users[userId]:
                recommend[mv] = 0
                for been in mv_list[:4]:
                    if mv != been:
                        simuliar = sim_cosine(mv_matrix,mv,been)
                        recommend[mv] = max(recommend[mv],simuliar)
        recommend = sorted(recommend, key=lambda key: recommend[key], reverse=True)

        mv_remd = recommend[:10]
    
    print("The recommended movies:")
    n = 1
    for i in item_data(mv_remd):
        mv_tp = ''
        for t in i[3:]:
            mv_tp += t
            mv_tp += ' '
        print("""
                ------------ {}.{} ------------
                Title : {}
                Release date: {}
                URL : {}
                Type: {}
                """.format(n, i[0], i[0], i[1], i[2], mv_tp))
        n += 1
    
    return evaluation(mv_remd,userId)

In [14]:
def predict_rating(userId,movieId):
    userlist = movies[movieId]
    simsums = 0.0
    predict = 0.0
    
    for u in userlist:
        sim = sim_cosine(users, u, userId)
        simsums += sim
        
    if simsums == 0:
        predict = 4
    else:
        for u in userlist:
            sim = sim_cosine(users, u, userId)
            percent = sim / simsums
            predict += percent * float(users[u][movieId])

    return predict

In [15]:
def evaluation(recommend,userId):
    i = 0
    
    for t in recommend:
        if t in test[userId]:
             i += 1
    if len(recommend) < len(test[userId]):
        return i/len(recommend)
    else:
        return i/len(test[userId])

In [16]:
def item_data(remd):
    
    item_list = []
    for i in remd:
        item_list.append(item[i])
    
    return item_list  

In [17]:
mv_recommend('44')

The recommended movies:

                ------------ 1.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 2.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 3.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)
                Type: Action Comedy Drama 
                

                ------------ 4.Copycat (1995) ------------
                Title : Copycat (1995)
                Release date: 01-Ja

0.3

In [18]:
iilist = []
for i in test:
    iilist.append(mv_recommend(i))

The recommended movies:

                ------------ 1.Postino, Il (1994) ------------
                Title : Postino, Il (1994)
                Release date: 01-Jan-1994
                URL : http://us.imdb.com/M/title-exact?Postino,%20Il%20(1994)
                Type: Drama Romance 
                

                ------------ 2.Fargo (1996) ------------
                Title : Fargo (1996)
                Release date: 14-Feb-1997
                URL : http://us.imdb.com/M/title-exact?Fargo%20(1996)
                Type: Crime Drama Thriller 
                

                ------------ 3.My Best Friend's Wedding (1997) ------------
                Title : My Best Friend's Wedding (1997)
                Release date: 20-Jun-1997
                URL : http://us.imdb.com/M/title-exact?My+Best+Friend%27s+Wedding+(1997)
                Type: Comedy Romance 
                

                ------------ 4.Contact (1997) ------------
                Title : Contact (1997)
         

                

                ------------ 8.Birdcage, The (1996) ------------
                Title : Birdcage, The (1996)
                Release date: 08-Mar-1996
                URL : http://us.imdb.com/M/title-exact?Birdcage,%20The%20(1996)
                Type: Comedy 
                

                ------------ 9.Apollo 13 (1995) ------------
                Title : Apollo 13 (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Apollo%2013%20(1995)
                Type: Action Drama Thriller 
                

                ------------ 10.Batman Forever (1995) ------------
                Title : Batman Forever (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Batman%20Forever%20(1995)
                Type: Action Adventure Comedy Crime 
                
The recommended movies:

                ------------ 1.Toy Story (1995) ------------
                Title : Toy

The recommended movies:

                ------------ 1.Toy Story (1995) ------------
                Title : Toy Story (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
                Type: Animation Children's Comedy 
                

                ------------ 2.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 3.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 4.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release 

The recommended movies:

                ------------ 1.Toy Story (1995) ------------
                Title : Toy Story (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
                Type: Animation Children's Comedy 
                

                ------------ 2.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 3.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 4.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release 

The recommended movies:

                ------------ 1.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 2.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 3.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)
                Type: Action Comedy Drama 
                

                ------------ 4.Copycat (1995) ------------
                Title : Copycat (1995)
                Release date: 01-Ja

...

The recommended movies:

                ------------ 1.Toy Story (1995) ------------
                Title : Toy Story (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
                Type: Animation Children's Comedy 
                

                ------------ 2.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 3.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 4.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release 

The recommended movies:

                ------------ 1.Toy Story (1995) ------------
                Title : Toy Story (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
                Type: Animation Children's Comedy 
                

                ------------ 2.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 3.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 4.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release 

The recommended movies:

                ------------ 1.Toy Story (1995) ------------
                Title : Toy Story (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
                Type: Animation Children's Comedy 
                

                ------------ 2.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 3.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 4.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release 

The recommended movies:

                ------------ 1.Toy Story (1995) ------------
                Title : Toy Story (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
                Type: Animation Children's Comedy 
                

                ------------ 2.GoldenEye (1995) ------------
                Title : GoldenEye (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
                Type: Action Adventure Thriller 
                

                ------------ 3.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 4.Get Shorty (1995) ------------
                Title : Get Shorty (1995)
                Release 

The recommended movies:

                ------------ 1.Four Rooms (1995) ------------
                Title : Four Rooms (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)
                Type: Thriller 
                

                ------------ 2.Copycat (1995) ------------
                Title : Copycat (1995)
                Release date: 01-Jan-1995
                URL : http://us.imdb.com/M/title-exact?Copycat%20(1995)
                Type: Crime Drama Thriller 
                

                ------------ 3.Belle de jour (1967) ------------
                Title : Belle de jour (1967)
                Release date: 01-Jan-1967
                URL : http://us.imdb.com/M/title-exact?Belle%20de%20jour%20(1967)
                Type: Drama 
                

                ------------ 4.Exotica (1994) ------------
                Title : Exotica (1994)
                Release date: 01-Jan-1994
       

In [20]:
print('{:.2%}'.format(sum(iilist)/len(iilist)))

13.31%
